# Optimisation du pas de temps

## Cadre de l'étude

Ce travail a pour objectif l'étude de l'influence du choix du pas de temps dans les méthodes de résolution d'équations différentielles vues en cours.
Plus exactement, on se propose de résoudre des problèmes de la forme : $ x' = f(x) $ avec $x : \mathbb R^n \longrightarrow \mathbb R^m $ et $f : \mathbb R^m \longrightarrow \mathbb R^m $

L'étude s'est faite en deux temps : tout d'abord nous avons considéré un pas de temps fixe et analysé son influence sur les différents schémas numériques. Nous avons également étudié l'importance de l'ordre de ces derniers : en quoi un schéma d'ordre $n$ est-il plus performant qu'un schéma d'ordre inférieur ?
Ensuite nous nous sommes intéressés au cas où le pas de temps est variable lors de la résolution de l'équation. Un programme nous a été fournis, nous devions le comprendre et l'illustrer dans des cas pertinents

## Imports de modules

Nous avons décidé d'insérer ici, une bonne fois pour toutes, les modules python que nous avons utilisé tout au long de l'étude

In [2]:
import math as ma
from numpy import *

ModuleNotFoundError: No module named 'numpy'

# Pas de temps fixe

Dans cette partie, nous donnerons nos implémentations de schémas numériques classiques (Euler, Runge-Kutta) et nous analyserons l'influence du choix du pas de temps sur ces derniers

### Euler

#### Cas explicite

Cet algorithme est des plus classiques, l'utilisation éventuelle de fonctions agissant sur des espaces de dimensions non réduite à 1 implique la manipulation de vecteurs **numpy**

In [3]:
def solve_euler_explicit(f, x0, t0, dt, t_tot=2):

    ''' Cette fonction renvoie la solution à l'équation différentielle dx/dt = f(x) 
    avec la méthode d'Euler explicite, pour une durée de 1 seconde'''
    
    N = int(ma.floor(t_tot/dt))
    t,x = t0,1*x0    # Donne le temps actuel et la position
    T = [t]  # Liste de temps
    X = array(x)  # Liste de positions
    for _ in range(0,N):
        t += dt    # Nouveau temps
        x += dt*f(x)  # Nouvelle position
        T.append(t)
        X = concatenate((X,x),axis = 1)

    return T, X

#### Cas implicite

Cet algorithme utilise le **théorème de Banach**, il faut donc choisir à partir de quand on considère le point fixe recherché avec la fonction **phi** atteint. Nous avons pris le partie de faire 100 itérations de **phi** puis ensuite d'étudier la variation relative en norme de deux itérations consécutives, en s'arrêtant lorsque cette dernière devient inférieure à 0.1%

In [ ]:
def euler_implicite(f, x0, t0, dt, t_tot = 2):
    N = int(ma.floor(t_tot/dt))  # Number of iterations
    t,x = t0,1*x0           # Current time & current position
    X = array(x)            # Liste de positions
    T = [t]                 # Liste de temps

    def phi(approx):
        return x + dt*f(approx) 
    for _ in range(N):
        t += dt
        approx_pos = phi(x)
        x = next_step(approx_pos, phi)
        T.append(t)
        X = concatenate((X,x),axis = 1)

    return T,X

def next_step(approx_pos,phi):
    for _ in range(100):
        approx_pos = phi(1*approx_pos)
    temp = phi(approx_pos)
    while linalg.norm(temp-approx_pos)/linalg.norm(approx_pos) > 0.001 :
        approx_pos = 1*temp
        temp = 1*phi(approx_pos)
    return approx_pos

### Runge-Kutta

#### Ordre 2

In [ ]:
def Runge_Kutta_2(f, x0, t0, dt, t_tot = 2):

    N = int(ma.floor(t_tot/dt))  # Number of iterations
    t,x = t0,1*x0                # Current time & current position
    X = array(x)                 # Liste de positions
    T = [t]                      # Liste de temps

    for k in range(N):
        t += dt
        middle_pos = x + dt/2*f(x)
        x += dt*f(middle_pos)

        T.append(t)
        X = concatenate((X,x),axis = 1)

    return T,X

#### Ordre 4

In [ ]:
def Runge_Kutta_4(f, x0, t0, dt, t_tot = 2):
    # PAS FINI
    N = int(ma.floor(t_tot/dt))  # Number of iterations
    t,x = t0,1*x0                # Current time & current position
    X = array(x)                 # Liste de positions
    T = [t]                      # Liste de temps

    for k in range(N):
        t += dt
        middle_pos_1 = x + dt/2*f(x)
        x += dt*f(middle_pos)

        T.append(t)
        X = concatenate((X,x),axis = 1)

    return T,X

### Résultat de tests

Nous avons décidé de comparer à la fois les méthodes entre elles et l'influence des pas de temps, afin de gagner en concision et en clarté.

#### Fonction exponentielle

#### Fonction carrée

#### Fonction cosinus

On observe que ces schémas numériques sont tous **consistants** : l'erreur maximale entre la solution approximée et la vraie solution tend vers 0 lorsque le pas de temps diminue aussi vers 0. C'est une propriété essentielle pour n'importe quel schéma numérique.
On voit de plus ici que nos implémentations fonctionnent pour des cas scalaires (exponentielle et carrée) et vectoriels (cosinus). Il est cependant à noter que les méthodes d'Euler semblent moins efficaces que celles de Runge-Kutta d'ordre supérieur à 1.


## Pas de temps variable

Maintenant que nous avons étudié l'influence du pas de temps, nous voulons optimiser le choix de ce dernier en fonction des situations. En effet une zone "sensible" où la fonction varie beaucoup nécessite un pas plus petit qu'une zone totalement linéaire. C'est la raison expliquant la variation possible du pas de temps.
L'algorithme suivant a été codé dans ce sens : on garde un pas "d'enregistrement" constant, c'est-à-dire que la solution reste échantillonnée sur des intervalles de longueue fixe $dt_{max}$, mais un autre pas est introduit : le pas **interne** $p_i$
Ce pas interne détermine les sous-intervalles sur lesquelles nous font les intégrations prévues dans la méthode d'Euler explicite. Il est choisi de manière à garder l'erreur d'approximation entre deux échantillonages inférieure à un certain seuil $T_{rel}$ : en effet, si le pas de temps interne $p_i$ donne lieu à une trop grande erreur d'approximation (i.e si $ ||x_{back} - x_{next}||$ est grand) on réduit ce pas, et vice-versa.